In [2]:
import functions.uap as uap
from sklearn.model_selection import train_test_split

target = 'target'
df_complete = uap.get_data()

In [6]:
df = uap.select_and_rename_columns(df_complete, target)
y = df[target]
X = df.drop(target, axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [2]:
# from imblearn.pipeline import Pipeline
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet, LinearRegression, Lasso, Ridge
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures  
import numpy as np
from sklearn.ensemble import RandomForestClassifier

param_elastic_default = {
    'alpha': [1.0],
    'copy_X': [True],
    'fit_intercept': [True],
    'l1_ratio': [0.5],
    'max_iter': [1000],
    'positive': [False],
    'precompute': [False],
    'random_state': [None],
    'selection': ['cyclic'],
    'tol': [0.0001],
    'warm_start': [False]
}
param = [
    {
        'imputer_simple__add_indicator': [True, False],
        # 'model_ridge__alpha': np.arange(0.001, 0.02, 0.0025),
        # 'model_lasso__alpha': np.arange(0.01, 1, 0.01),
        # 'model_elastic__alpha': np.arange(0.001, 0.02, 0.0025),
        # 'model_elastic__l1_ratio': np.arange(0, 1, 0.025),
        # 'model_elastic__tol': np.arange(0.0001, 0.0002, 0.001),
        # 'polynom__degree': np.arange(2, 4, 1),
        # 'polynom__degree': [2, 3],
        # 'imputer_simple__strategy': ['mean', 'median', 'most_frequent'],
        # 'imputer_knn__n_neighbors': [5],
        # 'imputer_knn__n_neighbors': np.arange(5, 100, 10),
        # 'imputer_knn__weights': ['uniform', 'distance'],
    }
]

preprocessor = ColumnTransformer([
    # ('imputer_knn', KNNImputer(), X.columns),
    # ('imputer_iterative', IterativeImputer(add_indicator=True), X.columns),
    ('imputer_simple', SimpleImputer(), X.columns),
    # ('scaler', StandardScaler(), X.columns),
])

pipe = Pipeline([
    # ('preprocessor', preprocessor),
    ('imputer_simple', SimpleImputer()),
    # ('imputer_simple', SimpleImputer()),
    ('scaler', StandardScaler()),
    # ('scaler', MinMaxScaler()),
    ('polynom', PolynomialFeatures(degree=3)),
    # ('model_ridge', Ridge(max_iter=10000))
    # ('model_lasso', Lasso(max_iter=10000))
    ('model_linear', LinearRegression())
    # ('model_elastic', ElasticNet(max_iter=10000, selection='random', tol=0.0002))
])

# import timeit
# start = timeit.default_timer()
# imputer = KNNImputer(n_neighbors=20, add_indicator=True)
# imputer.fit(X_train)
# fitted = timeit.default_timer()
# X_train_transform = imputer.transform(X_train)
# fitted_train = timeit.default_timer()
# X_test_transform = imputer.transform(X_test)
# end = timeit.default_timer()
# print(f'fitted: {fitted-start:.2f}s, train: {fitted_train-fitted:.2f}s test: {end-fitted_train:.2f}s')


model = GridSearchCV(pipe, param_grid=param, cv=10, scoring='r2', verbose=1, n_jobs=-1)
# model = RandomizedSearchCV(pipe, param_distributions=param, cv=10, n_iter=10, scoring='r2', verbose=1, n_jobs=-1)
# model.fit(X_train_transform, y_train)
# uap.check_regression(model, X_train_transform, X_test_transform, y_train, y_test)
model.fit(X_train, y_train)
uap.check_regression(model, X_train, X_test, y_train, y_test)
model



Fitting 10 folds for each of 2 candidates, totalling 20 fits


KeyboardInterrupt: 

## Linear Regression

In [ ]:
# from imblearn.pipeline import Pipeline
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

param = [
    {
        'imputer_simple__add_indicator': [True, False],
    }
]

pipe = Pipeline([
    ('imputer_simple', SimpleImputer()),
    ('scaler', StandardScaler()),
    ('model_linear', LinearRegression())
])

model = GridSearchCV(pipe, param_grid=param, cv=10, scoring='r2', verbose=1, n_jobs=-1)
model.fit(X_train, y_train)
uap.check_regression(model, X_train, X_test, y_train, y_test)
model

Fitting 10 folds for each of 2 candidates, totalling 20 fits



|Score|Test|Train|
|:--|--:|--:|
|MAE|27.861|27.809|
|MSE|1507.028|1463.830|
|R² Score|0.315|0.333|


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('imputer_simple', SimpleImputer()),
                                       ('scaler', StandardScaler()),
                                       ('model_linear', LinearRegression())]),
             n_jobs=-1,
             param_grid=[{'imputer_simple__add_indicator': [True, False]}],
             scoring='r2', verbose=1)

## Polynomial linear regression

In [ ]:
# from imblearn.pipeline import Pipeline
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, PolynomialFeatures  

param = [
    {
        'imputer_simple__strategy': ['mean', 'median', 'most_frequent'],
        'imputer_simple__add_indicator': [True, False],
    }
]

pipe = Pipeline([
    ('imputer_simple', SimpleImputer()),
    ('scaler', StandardScaler()),
    ('polynom', PolynomialFeatures(degree=3)),
    ('model_linear', LinearRegression())
])

model = GridSearchCV(pipe, param_grid=param, cv=10, scoring='r2', verbose=1, n_jobs=-1)
model.fit(X_train, y_train)
uap.check_regression(model, X_train, X_test, y_train, y_test)
model

Fitting 10 folds for each of 6 candidates, totalling 60 fits



|Score|Test|Train|
|:--|--:|--:|
|MAE|25.425|24.814|
|MSE|1314.726|1206.369|
|R² Score|0.402|0.450|


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('imputer_simple', SimpleImputer()),
                                       ('scaler', StandardScaler()),
                                       ('polynom',
                                        PolynomialFeatures(degree=3)),
                                       ('model_linear', LinearRegression())]),
             n_jobs=-1,
             param_grid=[{'imputer_simple__add_indicator': [True, False],
                          'imputer_simple__strategy': ['mean', 'median',
                                                       'most_frequent']}],
             scoring='r2', verbose=1)

In [ ]:
# from imblearn.pipeline import Pipeline
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import numpy as np

param = [
    {
        'imputer_simple__add_indicator': [True, False],
        'model_elastic__alpha': np.arange(0.001, 0.02, 0.0025),
        'model_elastic__l1_ratio': np.arange(0, 1, 0.025),
    }
]

pipe = Pipeline([
    ('imputer_simple', SimpleImputer()),
    ('scaler', StandardScaler()),
    ('model_elastic', ElasticNet(max_iter=10000, selection='random', tol=0.0002))
])

model = GridSearchCV(pipe, param_grid=param, cv=10, scoring='r2', verbose=1, n_jobs=-1)
model.fit(X_train, y_train)
uap.check_regression(model, X_train, X_test, y_train, y_test)
model




Fitting 10 folds for each of 640 candidates, totalling 6400 fits


/home/boesi/lessons/2025-05-06-ds-ml-project/.venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.614e+07, tolerance: 9.611e+03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/home/boesi/lessons/2025-05-06-ds-ml-project/.venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.625e+07, tolerance: 9.718e+03 Linear regression models with null weight for the l1 regularization term are mor

KeyboardInterrupt: 

In [ ]:
# from imblearn.pipeline import Pipeline
from sklearn.pipeline import Pipeline
from sklearn.linear_model import HuberRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import numpy as np

param = [
    {
        'imputer_simple__add_indicator': [True, False],
        # 'model_huber__alpha': np.arange(0.001, 0.02, 0.0025),
        # 'model_huber__epsilon': np.arange(0, 1, 0.025),
    }
]

pipe = Pipeline([
    ('imputer_simple', SimpleImputer()),
    ('scaler', StandardScaler()),
    ('model_huber', HuberRegressor(max_iter=10000))
])

model = GridSearchCV(pipe, param_grid=param, cv=10, scoring='r2', verbose=1, n_jobs=-1)
model.fit(X_train, y_train)
uap.check_regression(model, X_train, X_test, y_train, y_test)
model





Fitting 10 folds for each of 2 candidates, totalling 20 fits



|Score|Test|Train|
|:--|--:|--:|
|MAE|27.102|27.138|
|MSE|1542.110|1506.573|
|R² Score|0.299|0.314|


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('imputer_simple', SimpleImputer()),
                                       ('scaler', StandardScaler()),
                                       ('model_huber',
                                        HuberRegressor(max_iter=10000))]),
             n_jobs=-1,
             param_grid=[{'imputer_simple__add_indicator': [True, False]}],
             scoring='r2', verbose=1)

In [ ]:
# from imblearn.pipeline import Pipeline
from sklearn.pipeline import Pipeline
from sklearn.linear_model import RANSACRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import numpy as np

param = [
    {
        'imputer_simple__add_indicator': [True, False],
    }
]

pipe = Pipeline([
    ('imputer_simple', SimpleImputer()),
    ('scaler', StandardScaler()),
    ('model_ransac', RANSACRegressor())
])

model = GridSearchCV(pipe, param_grid=param, cv=10, scoring='r2', verbose=1, n_jobs=-1)
model.fit(X_train, y_train)
uap.check_regression(model, X_train, X_test, y_train, y_test)
model

Fitting 10 folds for each of 2 candidates, totalling 20 fits



|Score|Test|Train|
|:--|--:|--:|
|MAE|31.769|32.120|
|MSE|2038.496|2049.877|
|R² Score|0.073|0.066|


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('imputer_simple', SimpleImputer()),
                                       ('scaler', StandardScaler()),
                                       ('model_ransac', RANSACRegressor())]),
             n_jobs=-1,
             param_grid=[{'imputer_simple__add_indicator': [True, False]}],
             scoring='r2', verbose=1)

In [ ]:
# from imblearn.pipeline import Pipeline
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

param = [
    {
        'imputer_simple__add_indicator': [True, False],
    }
]

pipe = Pipeline([
    ('imputer_simple', SimpleImputer()),
    ('scaler', StandardScaler()),
    ('model_rfr', RandomForestRegressor())
])

model = GridSearchCV(pipe, param_grid=param, cv=10, scoring='r2', verbose=1, n_jobs=-1)
model.fit(X_train, y_train)
uap.check_regression(model, X_train, X_test, y_train, y_test)
model

Fitting 10 folds for each of 2 candidates, totalling 20 fits



|Score|Test|Train|
|:--|--:|--:|
|MAE|23.573|8.708|
|MSE|1181.176|159.625|
|R² Score|0.463|0.927|


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('imputer_simple', SimpleImputer()),
                                       ('scaler', StandardScaler()),
                                       ('model_rfr', RandomForestRegressor())]),
             n_jobs=-1,
             param_grid=[{'imputer_simple__add_indicator': [True, False]}],
             scoring='r2', verbose=1)

In [ ]:
# from imblearn.pipeline import Pipeline
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

param = [
    {
        'imputer_simple__add_indicator': [True, False],
    }
]

pipe = Pipeline([
    ('imputer_simple', SimpleImputer()),
    ('scaler', StandardScaler()),
    ('model_rfr', GradientBoostingRegressor())
])

model = GridSearchCV(pipe, param_grid=param, cv=10, scoring='r2', verbose=1, n_jobs=-1)
model.fit(X_train, y_train)
uap.check_regression(model, X_train, X_test, y_train, y_test)
model

Fitting 10 folds for each of 2 candidates, totalling 20 fits



|Score|Test|Train|
|:--|--:|--:|
|MAE|25.346|24.387|
|MSE|1298.798|1168.828|
|R² Score|0.410|0.467|


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('imputer_simple', SimpleImputer()),
                                       ('scaler', StandardScaler()),
                                       ('model_rfr',
                                        GradientBoostingRegressor())]),
             n_jobs=-1,
             param_grid=[{'imputer_simple__add_indicator': [True, False]}],
             scoring='r2', verbose=1)

In [ ]:
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import numpy as np

param_default = {
    'objective': 'reg:squarederror',
    'base_score': None,
    'booster': None,
    'callbacks': None,
    'colsample_bylevel': None,
    'colsample_bynode': None,
    'colsample_bytree': None,
    'device': None,
    'early_stopping_rounds': None,
    'enable_categorical': False,
    'eval_metric': None,
    'feature_types': None,
    'feature_weights': None,
    'gamma': None,
    'grow_policy': None,
    'importance_type': None,
    'interaction_constraints': None,
    'learning_rate': None,
    'max_bin': None,
    'max_cat_threshold': None,
    'max_cat_to_onehot': None,
    'max_delta_step': None,
    'max_depth': None,
    'max_leaves': None,
    'min_child_weight': None,
    'missing': np.nan,
    'monotone_constraints': None,
    'multi_strategy': None,
    'n_estimators': None,
    'n_jobs': None,
    'num_parallel_tree': None,
    'random_state': None,
    'reg_alpha': None,
    'reg_lambda': None,
    'sampling_method': None,
    'scale_pos_weight': None,
    'subsample': None,
    'tree_method': None,
    'validate_parameters': None,
    'verbosity': None
}

param = [
    {
        'imputer_simple__add_indicator': [True, False],
        'model_rfr__n_estimators': [50, 100, 200, 500],
        'model_rfr__max_depth': [3, 6, 10],
        # 'model_rfr__max_leaves': [],
        'model_rfr__booster': ['gbtree', 'gblinear', 'dart'],
        # 'model_rfr__tree_method': []
    }
]

pipe = Pipeline([
    ('imputer_simple', SimpleImputer()),
    ('scaler', StandardScaler()),
    ('model_rfr', XGBRegressor())
])

y = df_complete[target]
X = df_complete.drop([target, 'target_aqi', 'target_health', 'target_min', 'target_max', 'target_count', 'target_variance', 'Date', 'Place_ID', 'Place_ID X Date'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train['windspeed_lag1'] = X_train['windspeed'].shift(1)

# model = GridSearchCV(pipe, param_grid=param, cv=10, scoring='r2', verbose=1, n_jobs=-1)
# model.fit(X_train, y_train)
# uap.check_regression(model, X_train, X_test, y_train, y_test)
# model

In [ ]:
# from imblearn.pipeline import Pipeline
from sklearn.pipeline import Pipeline
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import numpy as np

target = 'target'
df_selected = uap.select_and_rename_columns(df_complete, target, feateng=False, feat_dates=True, keep_all_feeatures=True, check_validity=False)
y = df_selected[target]
X = df_selected.drop([target], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# X_train['windspeed_lag1'] = X_train['windspeed'].shift(1)

param = [
    {
        'model_hgbr__max_leaf_nodes': np.arange(2, 200, 1),
        'model_hgbr__learning_rate': [0.01, 0.1, 0.2],
        'model_hgbr__max_features': np.linspace(0.1, 1),
        'model_hgbr__l2_regularization': np.linspace(0.0, 1),
        'model_hgbr__max_depth': np.arange(1, 10, 1),
    }
]

pipe = Pipeline([
    ('imputer_simple', SimpleImputer(add_indicator=True)),
    ('scaler', StandardScaler()),
    ('model_hgbr', HistGradientBoostingRegressor())
])


model = RandomizedSearchCV(pipe, param, n_iter=20, cv=10, scoring='r2', verbose=1, n_jobs=-1)
model.fit(X_train, y_train)
uap.check_regression(model, X_train, X_test, y_train, y_test)
model


Fitting 10 folds for each of 20 candidates, totalling 200 fits


ValueError: 
All the 200 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/home/boesi/lessons/2025-05-06-ds-ml-project/.venv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/boesi/lessons/2025-05-06-ds-ml-project/.venv/lib/python3.11/site-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/boesi/lessons/2025-05-06-ds-ml-project/.venv/lib/python3.11/site-packages/sklearn/pipeline.py", line 662, in fit
    self._final_estimator.fit(Xt, y, **last_step_params["fit"])
  File "/home/boesi/